En el presente Notebook se utilizarán los datos diarios de Demanda Nacional de Energía para obtener un modelo de predicción a partir de redes neuronales. Se mejorará el modelo, realizando pruebas con distintos parámetros, seleccionando el que presente una r2 más alta. Una vez seleccionado, se guardará para su utilización en el Notebook de comparación frente a Prophet.

In [53]:
import numpy as np
import pandas as pd
from prophet import Prophet
import pickle

from datetime import datetime

import json
from pandas import Timestamp

import requests
from pprint import pprint

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, SimpleRNN, Dropout

from sklearn.metrics import r2_score, accuracy_score, mean_absolute_error, mean_squared_error

from sklearn.preprocessing import MinMaxScaler

import kaleido
import itertools
from prophet.diagnostics import cross_validation

En primer lugar, cargamos los datos e inicializamos el mismo modelo que habíamos desarrollado en el Notebook previo en el que se probó con todos los conjuntos de datos disponibles.

In [27]:
demanda_nacional=pd.read_csv("demanda_nacional.csv")
demanda_nacional

,Fecha,Energia Consumida Mw/h
0,2011-01-01,605.99
1,2011-01-02,641.86
2,2011-01-03,801.30
3,2011-01-04,833.25
4,2011-01-05,803.48
...,...,...
4611,2023-08-17,705.51
4612,2023-08-18,706.09
4613,2023-08-19,664.26
4614,2023-08-20,648.87


In [28]:
demanda_nacional["Fecha"][4383]

'2023-01-01'

In [29]:
demanda_nacional["Fecha"] = demanda_nacional["Fecha"].apply(lambda x : datetime.strptime(x,"%Y-%m-%d"))


In [5]:
# Preprocesamiento

datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
datos = datos.reshape(-1,1)

scaler = MinMaxScaler()
datos = scaler.fit_transform(datos)

datos.shape

(4616, 1)

In [6]:
T = 20 # Segmentos
X = list()
Y = list()

for t in range(len(datos) - T):
    
    # Toma valores de X de t en t con stride de 1
    x = datos[t : t + T]
    X.append(x)
    
    # Toma los valores de t en t
    y_ = datos[t + T]
    Y.append(y_)

# Transformamos a np.array y ajustamos las dimensiones
# Para entrar en el modelo debe de tener 3 dimensiones
X = np.array(X).reshape(-1, T, 1)
Y = np.array(Y)
N = len(X)

print(f"X: {X.shape}\ty: {Y.shape}")

X: (4596, 20, 1)	y: (4596, 1)


In [7]:
#Creamos el modelo

model = Sequential()

model.add(Input(shape = (T, 1)))

model.add(SimpleRNN(units = 200, activation = "relu"))

model.add(Dense(units = 60, activation="relu")) 
model.add(Dense(units = 30, activation="relu")) 
model.add(Dense(units = 15, activation="relu")) 
model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mse")

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 200)               40400     
                                                                 
 dense (Dense)               (None, 60)                12060     
                                                                 
 dense_1 (Dense)             (None, 30)                1830      
                                                                 
 dense_2 (Dense)             (None, 15)                465       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 54771 (213.95 KB)
Trainable params: 54771 (213.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Entrenamos el modelo

history = model.fit(x = X[:4363],
                        y = Y[:4363],
                        validation_data = (X[4363:], Y[4363:]),
                        epochs = 100)

Epoch 1/100
137/137 [==============================] - 4s 12ms/step - loss: 0.0194 - val_loss: 0.0041
Epoch 2/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0041 - val_loss: 0.0034
Epoch 3/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0035 - val_loss: 0.0029
Epoch 4/100
137/137 [==============================] - 2s 11ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 5/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0029 - val_loss: 0.0023
Epoch 6/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0030 - val_loss: 0.0024
Epoch 7/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0029 - val_loss: 0.0029
Epoch 8/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 9/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 10/100
137/137 [==============================] - 1s 10ms/step - loss: 0.002

137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0020
Epoch 81/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 82/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0023
Epoch 83/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0014 - val_loss: 0.0022
Epoch 84/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 85/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0014 - val_loss: 0.0022
Epoch 86/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0013 - val_loss: 0.0022
Epoch 87/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0013 - val_loss: 0.0019
Epoch 88/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0014 - val_loss: 0.0019
Epoch 89/100
137/137 [==============================] - 2s 14ms/step - loss: 0.0014 - 

In [9]:
# "1 - Step Predictions"

# Toma un valor y predice el siguiente

validation_target = Y[4363:]
validation_predictions = list()

i = -233

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
    i += 1
    
    validation_predictions.append(p)

1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 31ms/step


In [10]:
yhat_modelo=np.array(validation_predictions)

yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

target = np.array(validation_target)

target = scaler.inverse_transform(target.reshape(-1,1))

In [11]:
r2_score(target, yhat_modelo)

0.9181963499762459

### TUNING


Para mejorar el modelo, es posible hacer cambios en varias partes. En primer lugar, se realizarán pruebas añadiendo más capas a la red neuronal para ver como afectan. 

In [30]:
# Preprocesamiento

datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
datos = datos.reshape(-1,1)

scaler = MinMaxScaler()
datos = scaler.fit_transform(datos)

T = 20 # Segmentos
X = list()
Y = list()

for t in range(len(datos) - T):
    
    # Toma valores de X de t en t con stride de 1
    x = datos[t : t + T]
    X.append(x)
    
    # Toma los valores de t en t
    y_ = datos[t + T]
    Y.append(y_)

# Transformamos a np.array y ajustamos las dimensiones
# Para entrar en el modelo debe de tener 3 dimensiones
X = np.array(X).reshape(-1, T, 1)
Y = np.array(Y)
N = len(X)

#Creamos el modelo

model = Sequential()

model.add(Input(shape = (T, 1)))

model.add(SimpleRNN(units = 200, activation = "relu"))

model.add(Dense(units=120, activation = "relu"))
model.add(Dense(units = 60, activation="relu")) 
model.add(Dense(units = 30, activation="relu")) 
model.add(Dense(units = 15, activation="relu")) 
model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mse")

# Entrenamos el modelo

history = model.fit(x = X[:4363],
                        y = Y[:4363],
                        validation_data = (X[4363:], Y[4363:]),
                        epochs = 100)

# "1 - Step Predictions"

# Toma un valor y predice el siguiente

validation_target = Y[4363:]
validation_predictions = list()

i = -233

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
    i += 1
    
    validation_predictions.append(p)
    

yhat_modelo=np.array(validation_predictions)

yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

target = np.array(validation_target)

target = scaler.inverse_transform(target.reshape(-1,1))

r2_score(target, yhat_modelo)

Epoch 1/100
137/137 [==============================] - 3s 12ms/step - loss: 0.0324 - val_loss: 0.0080
Epoch 2/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0051 - val_loss: 0.0034
Epoch 3/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0040 - val_loss: 0.0034
Epoch 4/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0035 - val_loss: 0.0025
Epoch 5/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0035 - val_loss: 0.0025
Epoch 6/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0030 - val_loss: 0.0026
Epoch 7/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 8/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0028 - val_loss: 0.0023
Epoch 9/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 10/100
137/137 [==============================] - 1s 11ms/step - loss: 0.002

137/137 [==============================] - 1s 10ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 81/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0018 - val_loss: 0.0021
Epoch 82/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 83/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0015 - val_loss: 0.0019
Epoch 84/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 85/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 86/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 87/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 88/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 89/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0014 - 

1/1 [==============================] - 0s 31ms/step


0.9449088644114375

In [31]:
# Preprocesamiento

datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
datos = datos.reshape(-1,1)

scaler = MinMaxScaler()
datos = scaler.fit_transform(datos)

T = 20 # Segmentos
X = list()
Y = list()

for t in range(len(datos) - T):
    
    # Toma valores de X de t en t con stride de 1
    x = datos[t : t + T]
    X.append(x)
    
    # Toma los valores de t en t
    y_ = datos[t + T]
    Y.append(y_)

# Transformamos a np.array y ajustamos las dimensiones
# Para entrar en el modelo debe de tener 3 dimensiones
X = np.array(X).reshape(-1, T, 1)
Y = np.array(Y)
N = len(X)

#Creamos el modelo

model = Sequential()

model.add(Input(shape = (T, 1)))

model.add(SimpleRNN(units = 200, activation = "relu"))
model.add(Dense(units=150, activation="relu"))
model.add(Dense(units=120, activation = "relu"))
model.add(Dense(units = 60, activation="relu")) 
model.add(Dense(units = 30, activation="relu")) 
model.add(Dense(units = 15, activation="relu")) 
model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mse")

# Entrenamos el modelo

history = model.fit(x = X[:4363],
                        y = Y[:4363],
                        validation_data = (X[4363:], Y[4363:]),
                        epochs = 100)

# "1 - Step Predictions"

# Toma un valor y predice el siguiente

validation_target = Y[4363:]
validation_predictions = list()

i = -233

while len(validation_predictions) < len(validation_target):
    
    # Predice el siguiente valor de X[i]
    p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
    i += 1
    
    validation_predictions.append(p)
    

yhat_modelo=np.array(validation_predictions)

yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

target = np.array(validation_target)

target = scaler.inverse_transform(target.reshape(-1,1))

r2_score(target, yhat_modelo)

Epoch 1/100
137/137 [==============================] - 3s 13ms/step - loss: 0.0250 - val_loss: 0.0043
Epoch 2/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0038 - val_loss: 0.0036
Epoch 3/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 4/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0031 - val_loss: 0.0022
Epoch 5/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch 6/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0028 - val_loss: 0.0027
Epoch 7/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 8/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 9/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0025 - val_loss: 0.0022
Epoch 10/100
137/137 [==============================] - 1s 11ms/step - loss: 0.002

137/137 [==============================] - 1s 11ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 81/100
137/137 [==============================] - 2s 11ms/step - loss: 0.0018 - val_loss: 0.0024
Epoch 82/100
137/137 [==============================] - 2s 11ms/step - loss: 0.0016 - val_loss: 0.0023
Epoch 83/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 84/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0020 - val_loss: 0.0023
Epoch 85/100
137/137 [==============================] - 2s 13ms/step - loss: 0.0014 - val_loss: 0.0022
Epoch 86/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 87/100
137/137 [==============================] - 2s 12ms/step - loss: 0.0016 - val_loss: 0.0023
Epoch 88/100
137/137 [==============================] - 1s 11ms/step - loss: 0.0014 - val_loss: 0.0025
Epoch 89/100
137/137 [==============================] - 1s 10ms/step - loss: 0.0017 - 

1/1 [==============================] - 0s 20ms/step


0.9128204726342781

Al añadir una capa más, vemos que el modelo mejora ligeramente. Sin embargo, si se añade una siguiente capa, el r2 se reduce, lo que se puede deber al overfiting. 

Otras mejoras se pueden deber a modificaciones en el tamaño de los segmentos que coge el modelo para las predicciones (T) (se harán pruebas para 10, 20 y 30 unidades), el optimizador utilizado (adam o sdg) y las épocas. Para ello, se automatiza el entrenamiento del modelo.

### Automatización mejor modelo 


In [32]:
param_grid = {"optimizers": ["adam", "sgd"],
              "epochs_": [50, 70, 100, 120, 150]}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
r2_list = list()
modelos_list = list()
T_list = list()

In [33]:
all_params

[{'optimizers': 'adam', 'epochs_': 50},
 {'optimizers': 'adam', 'epochs_': 70},
 {'optimizers': 'adam', 'epochs_': 100},
 {'optimizers': 'adam', 'epochs_': 120},
 {'optimizers': 'adam', 'epochs_': 150},
 {'optimizers': 'sgd', 'epochs_': 50},
 {'optimizers': 'sgd', 'epochs_': 70},
 {'optimizers': 'sgd', 'epochs_': 100},
 {'optimizers': 'sgd', 'epochs_': 120},
 {'optimizers': 'sgd', 'epochs_': 150}]

### T = 10

In [34]:
for params in all_params:
    
    # Preprocesamiento

    datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
    datos = datos.reshape(-1,1)

    scaler = MinMaxScaler()
    datos = scaler.fit_transform(datos)

    T = 10 # Segmentos
    X = list()
    Y = list()

    for t in range(len(datos) - T):
    
        # Toma valores de X de t en t con stride de 1
        x = datos[t : t + T]
        X.append(x)
    
        # Toma los valores de t en t
        y_ = datos[t + T]
        Y.append(y_)

    # Transformamos a np.array y ajustamos las dimensiones
    # Para entrar en el modelo debe de tener 3 dimensiones
    X = np.array(X).reshape(-1, T, 1)
    Y = np.array(Y)
    N = len(X)
    
    #Creamos el modelo

    model = Sequential()

    model.add(Input(shape = (T, 1)))

    model.add(SimpleRNN(units = 200, activation = "relu"))
    model.add(Dense(units=120, activation = "relu"))
    model.add(Dense(units = 60, activation="relu")) 
    model.add(Dense(units = 30, activation="relu")) 
    model.add(Dense(units = 15, activation="relu")) 
    model.add(Dense(units = 1))

    model.compile(optimizer = params["optimizers"], loss = "mse")

    # Entrenamos el modelo

    history = model.fit(x = X[:4373],
                            y = Y[:4373],
                            validation_data = (X[4373:], Y[4373:]),
                            epochs = params["epochs_"])

    # "1 - Step Predictions"

    # Toma un valor y predice el siguiente

    validation_target = Y[4373:]
    validation_predictions = list()
    
    i = -233

    while len(validation_predictions) < len(validation_target):
    
        # Predice el siguiente valor de X[i]
        p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
        i += 1
    
        validation_predictions.append(p)
    

    yhat_modelo=np.array(validation_predictions)

    yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

    target = np.array(validation_target)

    target = scaler.inverse_transform(target.reshape(-1,1))

    r2_list.append(r2_score(target, yhat_modelo))
    modelos_list.append(model)
    T_list.append(T)

Epoch 1/50
137/137 [==============================] - 4s 11ms/step - loss: 0.0260 - val_loss: 0.0076
Epoch 2/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0052 - val_loss: 0.0038
Epoch 3/50
137/137 [==============================] - 1s 9ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 4/50
137/137 [==============================] - 1s 9ms/step - loss: 0.0036 - val_loss: 0.0030
Epoch 5/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0035 - val_loss: 0.0030
Epoch 6/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 7/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0031 - val_loss: 0.0027
Epoch 8/50
137/137 [==============================] - 1s 7ms/step - loss: 0.0031 - val_loss: 0.0028
Epoch 9/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0029 - val_loss: 0.0025
Epoch 10/50
137/137 [==============================] - 1s 8ms/step - loss: 0.0028 - val_loss: 0.002

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 19ms/step
Epoch 1/70
137/137 [==============================] - 3s 9ms/step - loss: 0.0232 - val_loss: 0.0052
Epoch 2/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 3/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0037 - val_loss: 0.0029
Epoch 4/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0034 - val_loss: 0.0028
Epoch 5/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0032 - val_loss: 0.0027
Epoch 6/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 7/70
137/137 [==============================] - 1s 9ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 8/70
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0029
Epoch 9/70
137/137 [==============================] - 1s 7ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 10/70
137/137 [===========================

1/1 [==============================] - 0s 27ms/step


1/1 [==============================] - 0s 20ms/step
Epoch 1/100
137/137 [==============================] - 3s 8ms/step - loss: 0.0322 - val_loss: 0.0090
Epoch 2/100
137/137 [==============================] - 1s 6ms/step - loss: 0.0058 - val_loss: 0.0044
Epoch 3/100
137/137 [==============================] - 1s 7ms/step - loss: 0.0040 - val_loss: 0.0041
Epoch 4/100
137/137 [==============================] - 1s 7ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 5/100
137/137 [==============================] - 1s 7ms/step - loss: 0.0035 - val_loss: 0.0033
Epoch 6/100
137/137 [==============================] - 1s 8ms/step - loss: 0.0034 - val_loss: 0.0031
Epoch 7/100
137/137 [==============================] - 1s 8ms/step - loss: 0.0032 - val_loss: 0.0028
Epoch 8/100
137/137 [==============================] - 1s 8ms/step - loss: 0.0031 - val_loss: 0.0027
Epoch 9/100
137/137 [==============================] - 1s 8ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 10/100
137/137 [=================

...

1/1 [==============================] - 0s 19ms/step
Epoch 1/150
137/137 [==============================] - 2s 8ms/step - loss: 0.0335 - val_loss: 0.0252
Epoch 2/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0184 - val_loss: 0.0221
Epoch 3/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0172 - val_loss: 0.0197
Epoch 4/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0162 - val_loss: 0.0172
Epoch 5/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0152 - val_loss: 0.0156
Epoch 6/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0143 - val_loss: 0.0152
Epoch 7/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0136 - val_loss: 0.0133
Epoch 8/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0130 - val_loss: 0.0127
Epoch 9/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0123 - val_loss: 0.0116
Epoch 10/150
137/137 [=================

Epoch 28/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0044 - val_loss: 0.0041
Epoch 29/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0043 - val_loss: 0.0038
Epoch 30/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0042 - val_loss: 0.0037
Epoch 31/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0041 - val_loss: 0.0036
Epoch 32/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0040 - val_loss: 0.0037
Epoch 33/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 34/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 35/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 36/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0038 - val_loss: 0.0033
Epoch 37/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0037

137/137 [==============================] - 1s 7ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 109/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 110/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 111/150
137/137 [==============================] - 1s 7ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 112/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 113/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 114/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 115/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 116/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - val_loss: 0.0028
Epoch 117/150
137/137 [==============================] - 1s 8ms/step - loss: 0.0030 - v

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 29ms/step


In [35]:
tuning_results_10 = pd.DataFrame(all_params)
tuning_results_10["T"] = T_list
tuning_results_10["r2"]=r2_list
tuning_results_10["Modelos"]=modelos_list

### T = 20

In [38]:
param_grid = {"optimizers": ["adam", "sgd"],
              "epochs_": [50, 70, 100, 120, 150]}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
r2_list = list()
modelos_list = list()
T_list = list()

In [39]:
for params in all_params:
    
    # Preprocesamiento

    datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
    datos = datos.reshape(-1,1)

    scaler = MinMaxScaler()
    datos = scaler.fit_transform(datos)

    T = 20 # Segmentos
    X = list()
    Y = list()

    for t in range(len(datos) - T):
    
        # Toma valores de X de t en t con stride de 1
        x = datos[t : t + T]
        X.append(x)
    
        # Toma los valores de t en t
        y_ = datos[t + T]
        Y.append(y_)

    # Transformamos a np.array y ajustamos las dimensiones
    # Para entrar en el modelo debe de tener 3 dimensiones
    X = np.array(X).reshape(-1, T, 1)
    Y = np.array(Y)
    N = len(X)
    
    #Creamos el modelo

    model = Sequential()

    model.add(Input(shape = (T, 1)))

    model.add(SimpleRNN(units = 200, activation = "relu"))
    model.add(Dense(units=120, activation = "relu"))
    model.add(Dense(units = 60, activation="relu")) 
    model.add(Dense(units = 30, activation="relu")) 
    model.add(Dense(units = 15, activation="relu")) 
    model.add(Dense(units = 1))

    model.compile(optimizer = params["optimizers"], loss = "mse")

    # Entrenamos el modelo

    history = model.fit(x = X[:4363],
                            y = Y[:4363],
                            validation_data = (X[4363:], Y[4363:]),
                            epochs = params["epochs_"])

    # "1 - Step Predictions"

    # Toma un valor y predice el siguiente

    validation_target = Y[4363:]
    validation_predictions = list()
    
    i = -233

    while len(validation_predictions) < len(validation_target):
    
        # Predice el siguiente valor de X[i]
        p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
        i += 1
    
        validation_predictions.append(p)
    

    yhat_modelo=np.array(validation_predictions)

    yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

    target = np.array(validation_target)

    target = scaler.inverse_transform(target.reshape(-1,1))

    r2_list.append(r2_score(target, yhat_modelo))
    modelos_list.append(model)
    T_list.append(T)

Epoch 1/50
137/137 [==============================] - 4s 14ms/step - loss: 0.0272 - val_loss: 0.0049
Epoch 2/50
137/137 [==============================] - 2s 12ms/step - loss: 0.0051 - val_loss: 0.0033
Epoch 3/50
137/137 [==============================] - 2s 12ms/step - loss: 0.0038 - val_loss: 0.0028
Epoch 4/50
137/137 [==============================] - 2s 12ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 5/50
137/137 [==============================] - 2s 14ms/step - loss: 0.0033 - val_loss: 0.0025
Epoch 6/50
137/137 [==============================] - 2s 15ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 7/50
137/137 [==============================] - 2s 15ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 8/50
137/137 [==============================] - 2s 16ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 9/50
137/137 [==============================] - 2s 15ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 10/50
137/137 [==============================] - 2s 14ms/step - loss: 0.0028 - val_lo

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 19ms/step
Epoch 1/70
137/137 [==============================] - 3s 14ms/step - loss: 0.0334 - val_loss: 0.0084
Epoch 2/70
137/137 [==============================] - 2s 12ms/step - loss: 0.0052 - val_loss: 0.0032
Epoch 3/70
137/137 [==============================] - 2s 14ms/step - loss: 0.0041 - val_loss: 0.0029
Epoch 4/70
137/137 [==============================] - 2s 14ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 5/70
137/137 [==============================] - 2s 15ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 6/70
137/137 [==============================] - 2s 14ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 7/70
137/137 [==============================] - 2s 14ms/step - loss: 0.0031 - val_loss: 0.0025
Epoch 8/70
137/137 [==============================] - 2s 13ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 9/70
137/137 [==============================] - 2s 13ms/step - loss: 0.0029 - val_loss: 0.0023
Epoch 10/70
137/137 [==================

1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 20ms/step
Epoch 1/100
137/137 [==============================] - 4s 22ms/step - loss: 0.0157 - val_loss: 0.0038
Epoch 2/100
137/137 [==============================] - 3s 20ms/step - loss: 0.0041 - val_loss: 0.0027
Epoch 3/100
137/137 [==============================] - 3s 21ms/step - loss: 0.0033 - val_loss: 0.0028
Epoch 4/100
137/137 [==============================] - 3s 23ms/step - loss: 0.0032 - val_loss: 0.0037
Epoch 5/100
137/137 [==============================] - 3s 23ms/step - loss: 0.0031 - val_loss: 0.0024
Epoch 6/100
137/137 [==============================] - 3s 22ms/step - loss: 0.0030 - val_loss: 0.0024
Epoch 7/100
137/137 [==============================] - 3s 21ms/step - loss: 0.0028 - val_loss: 0.0023
Epoch 8/100
137/137 [==============================] - 4s 27ms/step - loss: 0.0029 - val_loss: 0.0022
Epoch 9/100
137/137 [==============================] - 4s 26ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 10/100
137/137 [========

...

1/1 [==============================] - 0s 20ms/step
Epoch 1/150
137/137 [==============================] - 4s 23ms/step - loss: 0.0434 - val_loss: 0.0290
Epoch 2/150
137/137 [==============================] - 3s 21ms/step - loss: 0.0208 - val_loss: 0.0259
Epoch 3/150
137/137 [==============================] - 3s 22ms/step - loss: 0.0194 - val_loss: 0.0248
Epoch 4/150
137/137 [==============================] - 3s 24ms/step - loss: 0.0183 - val_loss: 0.0228
Epoch 5/150
137/137 [==============================] - 3s 24ms/step - loss: 0.0171 - val_loss: 0.0200
Epoch 6/150
137/137 [==============================] - 3s 22ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 7/150
137/137 [==============================] - 3s 23ms/step - loss: 0.0147 - val_loss: 0.0171
Epoch 8/150
137/137 [==============================] - 3s 23ms/step - loss: 0.0137 - val_loss: 0.0153
Epoch 9/150
137/137 [==============================] - 4s 26ms/step - loss: 0.0125 - val_loss: 0.0130
Epoch 10/150
137/137 [========

137/137 [==============================] - 5s 40ms/step - loss: 0.0073 - val_loss: 0.0063
Epoch 19/150
137/137 [==============================] - 5s 37ms/step - loss: 0.0070 - val_loss: 0.0060
Epoch 20/150
137/137 [==============================] - 5s 36ms/step - loss: 0.0067 - val_loss: 0.0059
Epoch 21/150
137/137 [==============================] - 5s 38ms/step - loss: 0.0064 - val_loss: 0.0055
Epoch 22/150
137/137 [==============================] - 6s 45ms/step - loss: 0.0061 - val_loss: 0.0052
Epoch 23/150
137/137 [==============================] - 5s 36ms/step - loss: 0.0058 - val_loss: 0.0049
Epoch 24/150
137/137 [==============================] - 6s 43ms/step - loss: 0.0056 - val_loss: 0.0047
Epoch 25/150
137/137 [==============================] - 5s 39ms/step - loss: 0.0054 - val_loss: 0.0046
Epoch 26/150
137/137 [==============================] - 6s 41ms/step - loss: 0.0052 - val_loss: 0.0043
Epoch 27/150
137/137 [==============================] - 6s 40ms/step - loss: 0.0050 - 

137/137 [==============================] - 9s 65ms/step - loss: 0.0033 - val_loss: 0.0026
Epoch 98/150
137/137 [==============================] - 10s 73ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 99/150
137/137 [==============================] - 9s 68ms/step - loss: 0.0033 - val_loss: 0.0026
Epoch 100/150
137/137 [==============================] - 10s 71ms/step - loss: 0.0033 - val_loss: 0.0026
Epoch 101/150
137/137 [==============================] - 10s 70ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 102/150
137/137 [==============================] - 9s 67ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 103/150
137/137 [==============================] - 9s 66ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 104/150
137/137 [==============================] - 9s 67ms/step - loss: 0.0032 - val_loss: 0.0025
Epoch 105/150
137/137 [==============================] - 9s 67ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 106/150
137/137 [==============================] - 9s 68ms/step - loss:

1/1 [==============================] - 0s 77ms/step


1/1 [==============================] - 0s 70ms/step


In [40]:
tuning_results_20 = pd.DataFrame(all_params)
tuning_results_20["T"] = T_list
tuning_results_20["r2"]=r2_list
tuning_results_20["Modelos"]=modelos_list

### T = 30

In [41]:
param_grid = {"optimizers": ["adam", "sgd"],
              "epochs_": [50, 70, 100, 120, 150]}

all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
r2_list = list()
modelos_list = list()
T_list = list()

In [42]:
for params in all_params:
    
    # Preprocesamiento

    datos = np.array(demanda_nacional["Energia Consumida Mw/h"])
    datos = datos.reshape(-1,1)

    scaler = MinMaxScaler()
    datos = scaler.fit_transform(datos)

    T = 30 # Segmentos
    X = list()
    Y = list()

    for t in range(len(datos) - T):
    
        # Toma valores de X de t en t con stride de 1
        x = datos[t : t + T]
        X.append(x)
    
        # Toma los valores de t en t
        y_ = datos[t + T]
        Y.append(y_)

    # Transformamos a np.array y ajustamos las dimensiones
    # Para entrar en el modelo debe de tener 3 dimensiones
    X = np.array(X).reshape(-1, T, 1)
    Y = np.array(Y)
    N = len(X)
    
    #Creamos el modelo

    model = Sequential()

    model.add(Input(shape = (T, 1)))

    model.add(SimpleRNN(units = 200, activation = "relu"))
    model.add(Dense(units=120, activation = "relu"))
    model.add(Dense(units = 60, activation="relu")) 
    model.add(Dense(units = 30, activation="relu")) 
    model.add(Dense(units = 15, activation="relu")) 
    model.add(Dense(units = 1))

    model.compile(optimizer = params["optimizers"], loss = "mse")

    # Entrenamos el modelo

    history = model.fit(x = X[:4353],
                            y = Y[:4353],
                            validation_data = (X[4353:], Y[4353:]),
                            epochs = params["epochs_"])

    # "1 - Step Predictions"

    # Toma un valor y predice el siguiente

    validation_target = Y[4353:]
    validation_predictions = list()
    
    i = -233

    while len(validation_predictions) < len(validation_target):
    
        # Predice el siguiente valor de X[i]
        p = model.predict(X[i].reshape(1, -1, 1))[0, 0]
        i += 1
    
        validation_predictions.append(p)
    

    yhat_modelo=np.array(validation_predictions)

    yhat_modelo = scaler.inverse_transform(yhat_modelo.reshape(-1,1))

    target = np.array(validation_target)

    target = scaler.inverse_transform(target.reshape(-1,1))

    r2_list.append(r2_score(target, yhat_modelo))
    modelos_list.append(model)
    T_list.append(T)

Epoch 1/50
137/137 [==============================] - 20s 98ms/step - loss: 0.0223 - val_loss: 0.0077
Epoch 2/50
137/137 [==============================] - 14s 103ms/step - loss: 0.0045 - val_loss: 0.0053
Epoch 3/50
137/137 [==============================] - 12s 90ms/step - loss: 0.0037 - val_loss: 0.0033
Epoch 4/50
137/137 [==============================] - 13s 97ms/step - loss: 0.0033 - val_loss: 0.0031
Epoch 5/50
137/137 [==============================] - 13s 95ms/step - loss: 0.0030 - val_loss: 0.0033
Epoch 6/50
137/137 [==============================] - 13s 94ms/step - loss: 0.0030 - val_loss: 0.0032
Epoch 7/50
137/137 [==============================] - 13s 94ms/step - loss: 0.0028 - val_loss: 0.0027
Epoch 8/50
137/137 [==============================] - 13s 93ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 9/50
137/137 [==============================] - 13s 98ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 10/50
137/137 [==============================] - 13s 96ms/step - loss: 0.00

1/1 [==============================] - 0s 78ms/step


1/1 [==============================] - 0s 67ms/step
Epoch 1/70
137/137 [==============================] - 19s 77ms/step - loss: 0.0347 - val_loss: 0.0175
Epoch 2/70
137/137 [==============================] - 11s 79ms/step - loss: 0.0106 - val_loss: 0.0054
Epoch 3/70
137/137 [==============================] - 11s 78ms/step - loss: 0.0042 - val_loss: 0.0040
Epoch 4/70
137/137 [==============================] - 11s 83ms/step - loss: 0.0038 - val_loss: 0.0032
Epoch 5/70
137/137 [==============================] - 12s 86ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 6/70
137/137 [==============================] - 13s 95ms/step - loss: 0.0032 - val_loss: 0.0024
Epoch 7/70
137/137 [==============================] - 13s 98ms/step - loss: 0.0030 - val_loss: 0.0022
Epoch 8/70
137/137 [==============================] - 13s 95ms/step - loss: 0.0029 - val_loss: 0.0024
Epoch 9/70
137/137 [==============================] - 13s 98ms/step - loss: 0.0028 - val_loss: 0.0028
Epoch 10/70
137/137 [=========

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 50ms/step
Epoch 1/100
137/137 [==============================] - 10s 47ms/step - loss: 0.0163 - val_loss: 0.0039
Epoch 2/100
137/137 [==============================] - 6s 47ms/step - loss: 0.0035 - val_loss: 0.0098
Epoch 3/100
137/137 [==============================] - 7s 49ms/step - loss: 0.0044 - val_loss: 0.0030
Epoch 4/100
137/137 [==============================] - 7s 54ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 5/100
137/137 [==============================] - 8s 57ms/step - loss: 0.0028 - val_loss: 0.0037
Epoch 6/100
137/137 [==============================] - 7s 52ms/step - loss: 0.0028 - val_loss: 0.0035
Epoch 7/100
137/137 [==============================] - 7s 53ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 8/100
137/137 [==============================] - 7s 54ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 9/100
137/137 [==============================] - 7s 55ms/step - loss: 0.0026 - val_loss: 0.0023
Epoch 10/100
137/137 [=======

...

1/1 [==============================] - 0s 28ms/step
Epoch 1/150
137/137 [==============================] - 5s 31ms/step - loss: 0.0330 - val_loss: 0.0206
Epoch 2/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0188 - val_loss: 0.0197
Epoch 3/150
137/137 [==============================] - 5s 33ms/step - loss: 0.0180 - val_loss: 0.0204
Epoch 4/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0173 - val_loss: 0.0197
Epoch 5/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0167 - val_loss: 0.0193
Epoch 6/150
137/137 [==============================] - 5s 34ms/step - loss: 0.0164 - val_loss: 0.0163
Epoch 7/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0156 - val_loss: 0.0170
Epoch 8/150
137/137 [==============================] - 4s 29ms/step - loss: 0.0148 - val_loss: 0.0150
Epoch 9/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0141 - val_loss: 0.0153
Epoch 10/150
137/137 [========

137/137 [==============================] - 5s 36ms/step - loss: 0.0060 - val_loss: 0.0089
Epoch 18/150
137/137 [==============================] - 4s 33ms/step - loss: 0.0055 - val_loss: 0.0050
Epoch 19/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0052 - val_loss: 0.0047
Epoch 20/150
137/137 [==============================] - 5s 34ms/step - loss: 0.0050 - val_loss: 0.0049
Epoch 21/150
137/137 [==============================] - 5s 35ms/step - loss: 0.0048 - val_loss: 0.0047
Epoch 22/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0046 - val_loss: 0.0041
Epoch 23/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0044 - val_loss: 0.0042
Epoch 24/150
137/137 [==============================] - 5s 36ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 25/150
137/137 [==============================] - 5s 34ms/step - loss: 0.0042 - val_loss: 0.0046
Epoch 26/150
137/137 [==============================] - 4s 31ms/step - loss: 0.0041 - 

137/137 [==============================] - 4s 32ms/step - loss: 0.0028 - val_loss: 0.0029
Epoch 97/150
137/137 [==============================] - 5s 34ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch 98/150
137/137 [==============================] - 5s 35ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch 99/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0028 - val_loss: 0.0041
Epoch 100/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0028 - val_loss: 0.0063
Epoch 101/150
137/137 [==============================] - 5s 34ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 102/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0028 - val_loss: 0.0025
Epoch 103/150
137/137 [==============================] - 4s 30ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 104/150
137/137 [==============================] - 4s 32ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 105/150
137/137 [==============================] - 5s 35ms/step - loss: 0.0

1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 32ms/step


In [43]:
tuning_results_30 = pd.DataFrame(all_params)
tuning_results_30["T"] = T_list
tuning_results_30["r2"]=r2_list
tuning_results_30["Modelos"]=modelos_list

Unimos ahora los 3 Dataframes para obtener los mejores parámetros y con ellos, el mejor modelo.

In [44]:
tuning_results = pd.concat([tuning_results_10, tuning_results_20, tuning_results_30], axis = 0, ignore_index = True)

In [45]:
tuning_results.sort_values("r2", ascending = False)

,optimizers,epochs_,T,r2,Modelos
21,adam,70,30,0.939899,<keras.src.engine.sequential.Sequential object...
23,adam,120,30,0.937481,<keras.src.engine.sequential.Sequential object...
10,adam,50,20,0.933759,<keras.src.engine.sequential.Sequential object...
1,adam,70,10,0.933518,<keras.src.engine.sequential.Sequential object...
11,adam,70,20,0.931913,<keras.src.engine.sequential.Sequential object...
22,adam,100,30,0.931835,<keras.src.engine.sequential.Sequential object...
12,adam,100,20,0.929100,<keras.src.engine.sequential.Sequential object...
0,adam,50,10,0.926530,<keras.src.engine.sequential.Sequential object...
2,adam,100,10,0.926410,<keras.src.engine.sequential.Sequential object...
28,sgd,120,30,0.925636,<keras.src.engine.sequential.Sequential object...


In [46]:
tuning_results.to_csv("Tuning 1 step - Demanda.csv", index = False)

In [52]:
tuning_results["Modelos"][21]

In [55]:
#Se guarda el mejor modelo para luego utililzarlo en el Notebook de comparación con Prophet y Multiple Step.

with open(file = "mejor_modelo_1STEP.pkl", mode = "bw") as file:
    pickle.dump(tuning_results["Modelos"][21], file)